<a href="https://colab.research.google.com/github/heosuab/Face_detection_and_mosaic/blob/main/Face_detection_Mosaic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install face_recognition opencv-python matplotlib
import cv2, os
import face_recognition as fr
from IPython.display import Image, display
from matplotlib import pyplot as plt

In [ ]:
import math
from sklearn import neighbors
import os
import os.path
import pickle
from pathlib import Path
from PIL import Image, ImageDraw, ImageFilter
import face_recognition as fr
from  face_recognition.face_recognition_cli import image_files_in_folder

ALLOWED_EXTENSIONS = {'png', 'jpg', 'jpeg'}

In [ ]:
#    Trains a k-nearest neighbors classifier for face recognition.

X = []
y = []
train_dir = '/content/drive/MyDrive/Colab_Notebooks/train_dir'
model_path = '/content/drive/MyDrive/Colab_Notebooks/model_save'
model_save_path = Path(model_path) / 'model_save.pkl'
n_neighbors = None
knn_algo = 'ball_tree'
verbose = False
count = 0
# Loop through each person in the training set
for class_dir in os.listdir(train_dir):
   if not os.path.isdir(os.path.join(train_dir, class_dir)):
       continue
     # Loop through each training image for the current person
   for img_path in image_files_in_folder(os.path.join(train_dir, class_dir)):
        count+=1
        image = fr.load_image_file(img_path)
        face_bounding_boxes = fr.face_locations(image)

        if len(face_bounding_boxes) != 1:
           # If there are no people (or too many people) in a training image, skip the image.
           if verbose:
                print("Image {} not suitable for training: {}".format(img_path, "Didn't find a face" if len(face_bounding_boxes) < 1 else "Found more than one face"))
        else:
           # Add face encoding for current image to the training set
            X.append(fr.face_encodings(image, known_face_locations=face_bounding_boxes)[0])
            y.append(class_dir)
 # Determine how many neighbors to use for weighting in the KNN classifier
if n_neighbors is None:
    n_neighbors = int(round(math.sqrt(len(X))))+7
    print(len(X), n_neighbors)
    if verbose:
        print("Chose n_neighbors automatically:", n_neighbors)
 # Create and train the KNN classifier
knn_clf = neighbors.KNeighborsClassifier(n_neighbors=n_neighbors, algorithm=knn_algo, weights='distance')
knn_clf.fit(X, y)
# Save the trained KNN classifier
if model_save_path is not None:
   with open(model_save_path, 'wb') as f:
       pickle.dump(knn_clf, f)

In [ ]:
#    Recognizes faces in given image using a trained KNN classifier

img_path = '/content/drive/MyDrive/Colab_Notebooks/test_dir/ten5.jpg'
knn_clf = None
model_path = '/content/drive/MyDrive/Colab_Notebooks/model_save/model_save.pkl'
distance_threshold=0.32

if not os.path.isfile(img_path) or os.path.splitext(img_path)[1][1:] not in ALLOWED_EXTENSIONS:
    raise Exception("Invalid image path: {}".format(img_path))

# Load a trained KNN model (if one was passed in)
if knn_clf is None:
    with open(model_path, 'rb') as f:
            knn_clf = pickle.load(f)

# Load image file and find face locations
X_img = fr.load_image_file(img_path)
X_face_locations = fr.face_locations(X_img)

if len(X_face_locations) == 0:
   predictions = {}
else:
  # Find encodings for faces in the test iamge
  faces_encodings = fr.face_encodings(X_img, known_face_locations=X_face_locations)

  # Use the KNN model to find the best matches for the test face
  closest_distances = knn_clf.kneighbors(faces_encodings, n_neighbors=1)
  are_matches = [closest_distances[0][i][0] <= distance_threshold for i in range(len(X_face_locations))]
  predictions = set()
  for pred, loc, rec in zip(knn_clf.predict(faces_encodings), X_face_locations, are_matches):
    print(loc, pred)
    if rec:
      predictions_df = (pred, loc)
      predictions.add(predictions_df)

    else:
      predictions_df = ('unknown', loc)
      predictions.add(predictions_df)


In [ ]:
pil_image = Image.open(img_path).convert("RGB")
draw = ImageDraw.Draw(pil_image)

for name, (top, right, bottom, left) in predictions:
    # Draw a box around the face using the Pillow module
    draw.rectangle(((left, top), (right, bottom)), outline=(0, 0, 0))
    if name=='unknown':
      crop_image = pil_image.crop((left, top, right, bottom))
      blur_image = crop_image.filter(ImageFilter.GaussianBlur(radius=5))
      pil_image.paste(blur_image, (left, top))

    # There's a bug in Pillow where it blows up with non-UTF-8 text
    # when using the default bitmap font
    name = name.encode("UTF-8")

     # Draw a label with a name below the face
    text_width, text_height = draw.textsize(name)
    draw.rectangle(((left, bottom + text_height + 5), (right, bottom)), fill=(0, 0, 0), outline=(0, 0,0))
    draw.text(((left + right)/2 - text_width/2, bottom + text_height - 8), name, fill=(255, 255, 255, 255))

# Remove the drawing library from memory as per the Pillow docs
del draw

# Display the resulting image
display(pil_image)

pil_image.save('final_image.jpg')

In [ ]:
 closest_distances

In [ ]:
np.min(closest_distances, axis=1)

In [ ]:
np.mean(closest_distances, axis=1)

In [ ]:
path_def = '/content/drive/MyDrive/Colab_Notebooks/sua_test_dir'

for filename in os.listdir(path_def):
  file_path=os.path.join(path_def, filename)
  print(file_path)
  print(filename)